In [ ]:
#install and load package
install.packages("dagitty")
install.packages("ggdag")
library(dagitty)
library(ggdag)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(dagitty)


subGraph <- function(nodes, g) {
  edges_df <- edges(g)
  edges_sub <- edges_df[ edges_df$v %in% nodes & edges_df$w %in% nodes, ]

  dag_string <- paste(
    "dag {",
    paste(
      sprintf("%s -> %s;", edges_sub$v, edges_sub$w),
      collapse = " "
    ),
    "}"
  )
  dagitty(dag_string)
}

g <- dagitty("
dag {
  Z1 -> X1
  X1 -> D
  Z1 -> X2
  Z2 -> X3
  X3 -> Y
  Z2 -> X2
  X2 -> Y
  X2 -> D
  M  -> Y
  D  -> M
}
")

cat("\n-------------------------------\n")
cat("Pearl's Example DAG 'g':\n")
print(g)
cat("-------------------------------\n")

cat("\n2. Checking d-separation statements:\n")

cat("\n(A) 'Z2 _||_ Z1' (unconditional)?\n")
resA <- dseparated(g, "Z2", "Z1", character(0))
cat("dseparated(g, 'Z2','Z1', no conditioning) =>", resA, "\n")
if(resA) {
  cat("=> This means the DAG implies Z2 and Z1 should be unconditionally independent.\n")
} else {
  cat("=> This means the DAG implies some dependence between Z2 and Z1.\n")
}

cat("\n(B) 'X2 _||_ X3 | Z2'?\n")
resB <- dseparated(g, "X2", "X3", c("Z2"))
cat("dseparated(g, 'X2','X3', 'Z2') =>", resB, "\n")
if(resB) {
  cat("=> The DAG implies X2 and X3 are independent once we condition on Z2.\n")
} else {
  cat("=> The DAG implies X2 and X3 are dependent even after conditioning on Z2.\n")
}

print( impliedConditionalIndependencies(g) )

cat("\n\nScenario: only {D, Y, X2, M} are measured:\n\nas can be seen in the implied conditional independcies above, we still have  M _||_ X2 | D, so yes there are testable implications.\n\n")

cat("Scenario: only {D, Y, X2} are measured:\n\nas can be seen in the implied conditional independencies above, there are none that only include D, Y, and X2, so no.\n\n")


cat("Scenario: all variables except X2 measured:\n\nStill have Z1 _||_ Z2 for example, so yes there are testable implications.\n\n")


cat("Competing Model: 'g_alt' with arrow reversed X2 <- D\n")

g_alt <- dagitty("
dag {
  Z1 -> X1
  X1 -> D
  Z1 -> X2
  Z2 -> X3
  X3 -> Y
  Z2 -> X2
  X2 -> Y
  D  -> X2
  M  -> Y
  D  -> M
}
")

cat("Original DAG expects:\n  'Z1 _||_ D | (X1, X2)'\n")
origTest <- dseparated(g, "Z1", "D", c("X1","X2"))
cat("In original DAG: dseparated =>", origTest, "\n")

cat("\nIn the alternative DAG 'g_alt':\n  'Z1 _||_ D | (X1, X2)'?\n")
altTest <- dseparated(g_alt, "Z1", "D", c("X1","X2"))
cat("=>", altTest, "\n")

if(origTest && !altTest) {
  cat("Conclusion: A conditional independence test for 'Z1 _||_ D | X1, X2' can distinguish the two models.\n")
} else {
  cat("Conclusion: The same test does not show the difference as expected, or the direction reversed.\n")
}


-------------------------------
Pearl's Example DAG 'g':
dag {
D
M
X1
X2
X3
Y
Z1
Z2
D -> M
M -> Y
X1 -> D
X2 -> D
X2 -> Y
X3 -> Y
Z1 -> X1
Z1 -> X2
Z2 -> X2
Z2 -> X3
}
-------------------------------

2. Checking d-separation statements:

(A) 'Z2 _||_ Z1' (unconditional)?
dseparated(g, 'Z2','Z1', no conditioning) => TRUE 
=> This means the DAG implies Z2 and Z1 should be unconditionally independent.

(B) 'X2 _||_ X3 | Z2'?
dseparated(g, 'X2','X3', 'Z2') => TRUE 
=> The DAG implies X2 and X3 are independent once we condition on Z2.
D _||_ X3 | Z2
D _||_ X3 | X2, Z1
D _||_ X3 | X1, X2
D _||_ Y | M, X2, X3
D _||_ Y | M, X2, Z2
D _||_ Y | M, X2, Z1
D _||_ Y | M, X1, X2
D _||_ Z1 | X1, X2
D _||_ Z2 | X2, Z1
D _||_ Z2 | X1, X2
M _||_ X1 | D
M _||_ X2 | D
M _||_ X3 | Z2
M _||_ X3 | X2, Z1
M _||_ X3 | X1, X2
M _||_ X3 | D
M _||_ Z1 | X1, X2
M _||_ Z1 | D
M _||_ Z2 | X2, Z1
M _||_ Z2 | X1, X2
M _||_ Z2 | D
X1 _||_ X2 | Z1
X1 _||_ X3
X1 _||_ Y | M, X2, X3
X1 _||_ Y | M, X2, Z2
X1 _||_ Y | M, X2